In [ ]:
import numpy as np
import pandas as pd 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
from yellowbrick.classifier import ClassPredictionError
from sklearn.metrics import classification_report

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
# !pip install datasets

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyro-ppl 1.3.1 requires opt-einsum>=2.3.2, which is not installed.
spacy 2.1.4 requires jsonschema<3.1.0,>=2.6.0, but you have jsonschema 3.2.0 which is incompatible.


In [ ]:
from datasets import load_datasets

# Load Datasets

In [ ]:
train = load_dataset('turingbench/TuringBench',name='AA', split='train')
train = pd.DataFrame.from_dict(train)

test = load_dataset('turingbench/TuringBench',name='AA', split='test')
test = pd.DataFrame.from_dict(test)

valid = load_dataset('turingbench/TuringBench',name='AA', split='valid')
valid = pd.DataFrame.from_dict(valid)

# N-grams

In [ ]:
import nltk
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
train_ngrams = []
for i in train['Generation']:
    train_ngrams.append(extract_ngrams(str(i), 3))

In [ ]:
test_ngrams = []
for i in test['Generation']:
    test_ngrams.append(extract_ngrams(str(i), 3))

# SVM

In [ ]:
def classify(train, test, train_label, test_label):
            
        
#     X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    clf = svm.SVC(kernel='linear', gamma="auto")  
    v = TfidfVectorizer()

    
    train_corpus = train
    train_corpus = [" ".join(x) for x in train_corpus]

    test_corpus = test
    test_corpus = [" ".join(x) for x in test_corpus]
    
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    y_train = train_label
    y_test = test_label
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    print(classification_report(y_test, pred, labels = list(train_label.unique()),
                                digits=4))
    
    
    
    return y_test, pred

In [ ]:
# SVM 3-grams 

classify(train = train_ngrams, test = test_ngrams, 
             train_label = train['label'], test_label = test['label'])

# Random Forest

In [ ]:
def classify(train, test):
            
    
#     X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    X_train, X_test, y_train, y_test = list(train['Generation']), list(test['Generation']), list(train['label']), list(test['label'])
    
    clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
#     clf = LogisticRegression(solver = 'lbfgs',multi_class='auto')
    
    v = TfidfVectorizer()
    
    train_corpus = X_train
#     train_corpus = [[word.lower() for word in text.split()] for text in data]
#     train_corpus = [" ".join(x) for x in train_corpus]

    test_corpus = X_test
#     test_corpus = [" ".join(x) for x in test_corpus]

#     ax = axes.set_ylim([0,300])
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    LABEL = list(train['label'].unique())
    
    # Instantiate the classification model and visualizer
    visualizer = ClassPredictionError(clf, classes= LABEL)

    # Fit the training data to the visualizer
    visualizer.fit(train_vector, y_train)

    # Evaluate the model on the test data
    visualizer.score(test_vector, y_test)

    # Draw visualization
    visualizer.show()
    
    
    matrix = confusion_matrix(y_test, pred, labels = LABEL)
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, pred, labels = LABEL, digits=4))
    print('confusion matrix: ', mat)
    
    
    cm = confusion_matrix(y_test, pred, labels=LABEL)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

    ax.set_title('Confusion Matrix')


    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')

#     plt.xticks(rotation=70)

    ax.xaxis.set_ticklabels(LABEL)
    ax.yaxis.set_ticklabels(LABEL)
    plt.xticks(rotation=70)
    plt.yticks(rotation=0)



    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    print('F1:', F1)
    
    return clf, v

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
rd, v = classify(train = train, test = test)

# WriteprintsRFC

In [ ]:
train_writeprints = pd.read_csv('train_write.csv')
test_writeprints = pd.read_csv('test_write.csv')

In [ ]:
def classify_write(train, test, train_label, test_label):
            
        
#     X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
    
    train_corpus = train
#     train_corpus = [" ".join(x) for x in train_corpus]

    test_corpus = test
#     test_corpus = [" ".join(x) for x in test_corpus]
    
    train_vector = train_corpus
    test_vector = test_corpus
    
    y_train = train_label
    y_test = test_label
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    print(classification_report(y_test, pred, labels = list(train_label.unique()),
                                digits=4))
    
    
    
    return y_test, pred

In [ ]:
classify_write(train = train_writeprints.loc[:, train_writeprints.columns != 'label'],
              test = test_writeprints.loc[:, test_writeprints.columns != 'label'],
              train_label = list(train_writeprints['label']),
              test_label = list(test_writeprints['label']))